## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config2 import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Busselton,AU,-33.6500,115.3333,47.34,85,21,6.33,few clouds
1,1,Esil,KZ,51.9556,66.4042,66.45,36,5,6.17,clear sky
2,2,Provideniya,RU,64.3833,-173.3000,46.36,100,100,1.99,overcast clouds
3,3,Jalalabad,AF,34.4265,70.4515,84.38,49,0,2.84,clear sky
4,4,Albany,US,42.6001,-73.9662,92.03,54,28,4.00,scattered clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("Enter the desired minimum temperature you want to stay in? "))
max_temp = float(input("Enter the desired max temperature you want to stay in? "))

Enter the desired minimum temperature you want to stay in? 50
Enter the desired max temperature you want to stay in? 70


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_city_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_city_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Esil,KZ,51.9556,66.4042,66.45,36,5,6.17,clear sky
7,7,Huarmey,PE,-10.0681,-78.1522,67.30,71,12,11.68,few clouds
10,10,Hasaki,JP,35.7333,140.8333,68.40,90,100,13.62,overcast clouds
11,11,Torbay,CA,47.6666,-52.7314,69.76,76,100,20.71,overcast clouds
13,13,New Norfolk,AU,-42.7826,147.0587,50.90,73,100,1.54,overcast clouds
...,...,...,...,...,...,...,...,...,...,...
705,705,Ullapool,GB,57.8987,-5.1604,58.35,95,37,6.02,scattered clouds
706,706,Skelleftea,SE,64.7507,20.9528,51.01,100,0,3.44,clear sky
710,710,Bemidji,US,47.4736,-94.8803,65.05,91,100,14.97,overcast clouds
711,711,Kissidougou,GN,9.1848,-10.0999,69.69,97,90,6.26,light rain


In [6]:
# 4a. Determine if there are any empty rows.
preferred_city_df.isnull().sum()

City_ID                0
City                   0
Country                6
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [7]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_city_df = preferred_city_df.dropna()
preferred_city_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_city_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Esil,KZ,66.45,clear sky,51.9556,66.4042,
7,Huarmey,PE,67.30,few clouds,-10.0681,-78.1522,
10,Hasaki,JP,68.40,overcast clouds,35.7333,140.8333,
11,Torbay,CA,69.76,overcast clouds,47.6666,-52.7314,
13,New Norfolk,AU,50.90,overcast clouds,-42.7826,147.0587,
16,Maarianhamina,AX,57.06,clear sky,60.0973,19.9348,
19,Zaysan,KZ,59.58,clear sky,47.4667,84.8667,
20,East London,ZA,58.23,overcast clouds,-33.0153,27.9116,
21,Thompson,CA,57.36,broken clouds,55.7435,-97.8558,
23,Kasane,BW,66.49,clear sky,-17.8167,25.1500,


In [12]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index,row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6c. Get latitude and longitude from DataFrame
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index,"Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found.... skipping.")
        

Hotel not found.... skipping.
Hotel not found.... skipping.
Hotel not found.... skipping.
Hotel not found.... skipping.
Hotel not found.... skipping.
Hotel not found.... skipping.
Hotel not found.... skipping.
Hotel not found.... skipping.
Hotel not found.... skipping.
Hotel not found.... skipping.
Hotel not found.... skipping.
Hotel not found.... skipping.
Hotel not found.... skipping.
Hotel not found.... skipping.
Hotel not found.... skipping.
Hotel not found.... skipping.
Hotel not found.... skipping.
Hotel not found.... skipping.
Hotel not found.... skipping.
Hotel not found.... skipping.
Hotel not found.... skipping.
Hotel not found.... skipping.
Hotel not found.... skipping.
Hotel not found.... skipping.


In [16]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df = hotel_df[hotel_df["Hotel Name"] != ""]
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Esil,KZ,66.45,clear sky,51.9556,66.4042,Egorkino Hotel
7,Huarmey,PE,67.30,few clouds,-10.0681,-78.1522,Hotel Miramar
10,Hasaki,JP,68.40,overcast clouds,35.7333,140.8333,Inubosaki Kanko Hotel
11,Torbay,CA,69.76,overcast clouds,47.6666,-52.7314,Gallows Cove
13,New Norfolk,AU,50.90,overcast clouds,-42.7826,147.0587,The Shingles Riverside Cottages
...,...,...,...,...,...,...,...
705,Ullapool,GB,58.35,scattered clouds,57.8987,-5.1604,Argyll Hotel
706,Skelleftea,SE,51.01,clear sky,64.7507,20.9528,Quality Hotel Skellefteå Stadshotell
710,Bemidji,US,65.05,overcast clouds,47.4736,-94.8803,Hampton Inn & Suites Bemidji
711,Kissidougou,GN,69.69,light rain,9.1848,-10.0999,Hôtel Mantis


In [19]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [21]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>

</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [23]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
# 11b. Display the figure
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))